In [36]:
from keras_facenet import FaceNet
from mtcnn.mtcnn import MTCNN
from sklearn.metrics.pairwise import cosine_similarity
import cv2
from PIL import Image
import numpy as np

In [37]:
with open(r'C:\Users\Raum\Desktop\jec\code\experimental\array_of_frame\gg.npy', 'rb') as f:
    a = np.load(f)
a.shape

(1967,)

In [38]:
embedder = FaceNet()
detector = MTCNN()

In [43]:
def extraction_keras_lock(*file_name,show=None):
  if type(file_name) is tuple:
    file_name = [*file_name,]
  result = []
  for i in file_name:
    image = Image.open(i).convert('RGB')
    im_arr = np.array(image)
    N=0
    faces = detector.detect_faces(im_arr)
    x1,y1,width,height = faces[N]['box'] # faces[N]['box']
    x2,y2 = (x1+width),(y1+height)
    face = im_arr[y1:y2,x1:x2] # ตัดเอาเฉพาะใบหน้า
    image_face = Image.fromarray(face)
    image_face= image_face.resize((224,224))
    face_img = np.expand_dims(image_face,axis=0)
    result.append(embedder.embeddings(face_img))

  return result

In [117]:
def extraction_keras_all_cv(im_arr):
    position_crop = []
    embedding_faces = []
    position = []
    faces = detector.detect_faces(im_arr)
    
    if faces == []:
        return False

    for i in faces:
        x1,y1,width,height = i['box']
        x2,y2 = (x1+width),(y1+height)
        position_crop.append([y1,y2,x1,x2])
        position.append([x1,y1,x2,y2])

    # print(len(position_crop))
    for i in position_crop:
        # print(i)
        face = im_arr[i[0]:i[1],i[2]:i[3]]
        image_face = cv2.resize(im_arr,(224,224))
        faces_crops = np.expand_dims(image_face,axis=0)
        embedding_faces.append(embedder.embeddings(faces_crops))
        print(embedding_faces)
    return embedding_faces,position

In [118]:
image=cv2.imread(r'C:\Users\Raum\Desktop\jec\code\dataface\pitha.jpg')
im_arr = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
embedding_faces,position = extraction_keras_all_cv(im_arr)
# xx = extraction_keras_lock(r'C:\Users\Raum\Desktop\jec\code\dataface\Pita.jpg')
# extraction_keras_all_cv(im_arr)

[array([[ 0.02135664, -0.05255182, -0.0615859 ,  0.09021682, -0.0268083 ,
        -0.0043442 , -0.03667516,  0.04013048, -0.01611624, -0.01735125,
        -0.03381226,  0.00104054, -0.01560687, -0.05791557, -0.00658233,
        -0.04079702, -0.02450754, -0.01071068,  0.03536964, -0.11227509,
        -0.0880132 ,  0.03376059, -0.0206355 , -0.00746811, -0.03172487,
         0.02371309, -0.0248422 , -0.06844507,  0.01182395,  0.03667638,
        -0.04535117,  0.03018595,  0.06226763, -0.05343787, -0.05587618,
         0.03014533, -0.00389283,  0.01843305, -0.05150362,  0.05983732,
        -0.02052035,  0.02751451, -0.01183064, -0.03897709,  0.04919853,
         0.04143475,  0.0177028 ,  0.05720628, -0.05944101, -0.0193317 ,
         0.04709948,  0.04478164,  0.00855812,  0.02095119, -0.07316615,
         0.03417497,  0.06641663,  0.05832796,  0.0129271 , -0.04912737,
        -0.03803797, -0.00068919, -0.0115408 , -0.09121558, -0.04443774,
         0.09846161,  0.03278146,  0.0179865 , -0.

In [116]:
xx[0]

array([[ 6.18861243e-03,  2.83162481e-05, -5.23564406e-02,
         1.92748234e-02,  7.02566132e-02,  5.41722402e-02,
         3.77270505e-02,  4.65443768e-02,  1.12877544e-02,
         9.76526216e-02,  1.34116802e-02,  6.62989318e-02,
         3.41995060e-02,  7.99929053e-02,  7.05797970e-02,
         4.97564347e-03,  4.77358773e-02,  6.17558695e-02,
        -6.08610250e-02,  1.36749540e-02, -5.71898744e-02,
         3.20395231e-02,  7.05603734e-02,  3.84622887e-02,
        -2.25737598e-02,  6.10597152e-03, -3.48277278e-02,
         2.19291300e-02,  2.93812957e-02, -3.35963480e-02,
         2.94671338e-02, -1.87239386e-02,  7.40041863e-03,
         8.94995593e-03,  1.57014225e-02, -6.02718554e-02,
        -1.01966802e-02, -3.59052978e-02, -1.36765428e-02,
        -2.65306700e-03,  2.27476400e-03, -3.02061159e-02,
        -5.19685075e-03,  9.33301542e-03,  6.17287541e-03,
         6.53312057e-02,  7.21337199e-02,  7.57547515e-03,
         2.49742880e-03, -1.06402095e-02, -4.75350171e-0

array([[-0.13789493]], dtype=float32)

In [90]:
log_cosine_similarity = []

Detection_check = np.zeros((len(position)))
for j in range(len(xx)):

    for i in range(len(position)):
            similar = cosine_similarity(xx[j], embedding_faces[i])
            log_cosine_similarity.append(similar)
            if similar>0.4 and Detection_check[i] ==0 :
                Detection_check[i] = 1
                break

for i in range(len(position)):
    if Detection_check[i] == 1:
          pass
    else :
        censor_region = (position[i][0],position[i][1],position[i][2],position[i][3])

        censored_area = image[censor_region[1]:censor_region[3], censor_region[0]:censor_region[2]]

        censored_width, censored_height = censored_area.shape[1], censored_area.shape[0]


        pixel_size = 12
        censored_area = cv2.resize(censored_area, (censored_width // pixel_size, censored_height // pixel_size))
        censored_area = cv2.resize(censored_area, (censored_width, censored_height), interpolation=cv2.INTER_NEAREST)

        image[censor_region[1]:censor_region[3], censor_region[0]:censor_region[2]] = censored_area

log_cosine_similarity

[array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32),
 array([[-0.13789493]], dtype=float32)]

[array([[-0.09894437]], dtype=float32),
 array([[-0.09894437]], dtype=float32),
 array([[-0.09894437]], dtype=float32),
 array([[-0.09894437]], dtype=float32),
 array([[-0.09894437]], dtype=float32)]

In [68]:
cv2.imwrite('ggggg.jpg',image)

True